In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import warnings
import os
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot

import seaborn as sns
import tensorflow 
print(tensorflow.__version__)

2.4.1


In [19]:
# suppress display of warnings
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

In [ ]:
blog = pd.read_csv('/content/drive/MyDrive/Dataset - blogtext.csv')

In [ ]:
blog.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


In [ ]:
blog['age_group'] = blog['age'].apply(lambda x: '30s' if x>28 else ('10s' if x<20 else '20s'))

In [ ]:
blog.drop('age',axis=1, inplace= True)## dropping the age column

In [ ]:
blog_df= blog.sample(100000, random_state=2).reset_index(drop=True)

In [ ]:
## removing unnecessary columns which are blogger id, date:

blog_df.drop(['id','date'],axis=1,inplace=True)

In [ ]:
blog_df["target"] = blog_df.apply(lambda x : [x["gender"],x["age_group"],x["topic"],x["sign"]],axis =1)

In [ ]:
blog_df['len'] = blog_df['text'].apply(lambda x: len(x.split())) ## calculating the length of each row

In [ ]:
ind = blog_df[blog_df['len']==0].index

In [ ]:
blog_df.drop(index = ind, axis=0, inplace =True)

In [ ]:
blog_df.shape

(99534, 7)

##Text Preprocessing

In [ ]:
import re

In [ ]:
blog_df['text'] = blog_df['text'].apply(lambda x: str(x).lower()) ## converting to lower case

In [ ]:
blog_df['text'] = blog_df['text'].apply(lambda x: re.sub(r'https?:\/\/.\S+', " ",x)) ## removes the http: links

In [ ]:
print(blog_df['text'][2])

       i woke up very early on saturday morning, more early than i’d have hoped on a day off. i was planning on going to the beach, set up my chair and read some scripts for work and do some of my own work. i was going to a party late in the afternoon, so i wanted to get as much work in as possible so i could enjoy myself without feeling guilty. but since i had woken up at 5 in the morning, it was too dark and too chilly to head to the beach right away. and the fork in the road at the end of sunset canyon that leads onto the pch is tempting to someone like me, who adores the coastline here and is mystified by the landscape that spreads north of los angeles. if i had turned left then i would’ve been in santa monica and eventually in the marina. but if i turned right, then i’d be driving through malibu and beyond. i love what’s beyond malibu – every bit of it. so i turned right.   the pch is beautiful at five-thirty or six in the morning. the ocean smells the best at that time and there 

In [ ]:
blog_df['text']= blog_df['text'].apply(lambda x: re.sub(r"\b[^\s]+@[^\s]+[.][^\s]+\b", " ",x)) ## removing email id's

In [ ]:
blog_df['text'] = blog_df['text'].replace('urlLink',' ',regex=True) ## removing the word urlLink

In [ ]:
blog_df['text'] = blog_df['text'].apply(lambda x: re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x))

**Removing short forms**

In [ ]:
def short_form(text):
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "can not ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"youll", " you will ", text)
  text = re.sub(r"theyll", " they will ", text)
  text = re.sub(r"youve", " you have ", text)
  text = re.sub(r"theyve", " they have ", text)
  text = re.sub(r"cant", "can not ", text)
  text = re.sub(r"youre", "you are", text)
  text = re.sub(r"yup", "yes", text)
  text = re.sub(r"ohh", " ", text)
  text = re.sub(r"ahh", " ", text)
  text = re.sub(r"ahhh", " ", text)
  return text

In [ ]:
blog_df['text'] = blog_df['text'].apply(lambda x: short_form(x))

In [ ]:
blog_df.head()

,gender,topic,sign,text,age_group,target,len
0,male,Government,Pisces,the incompetence of transport plann...,30s,"[male, 30s, Government, Pisces]",354
1,male,Student,Libra,2/10/04 one more week till hun...,10s,"[male, 10s, Student, Libra]",766
2,female,indUnk,Aries,i woke up very early on saturday mornin...,20s,"[female, 20s, indUnk, Aries]",692
3,male,Technology,Leo,"ok, lots more pictures! urllink m...",20s,"[male, 20s, Technology, Leo]",23
4,male,indUnk,Aries,"happy birthday, mom! today is ...",30s,"[male, 30s, indUnk, Aries]",92


**Removing Special Characters**

In [ ]:
blog_df['text'] = blog_df['text'].replace('[^\w\s]',' ',regex= True)

In [ ]:
blog_df.head()

,gender,topic,sign,text,age_group,target,len
0,male,Government,Pisces,the incompetence of transport plann...,30s,"[male, 30s, Government, Pisces]",354
1,male,Student,Libra,2 10 04 one more week till hun...,10s,"[male, 10s, Student, Libra]",766
2,female,indUnk,Aries,i woke up very early on saturday mornin...,20s,"[female, 20s, indUnk, Aries]",692
3,male,Technology,Leo,ok lots more pictures urllink m...,20s,"[male, 20s, Technology, Leo]",23
4,male,indUnk,Aries,happy birthday mom today is ...,30s,"[male, 30s, indUnk, Aries]",92


** Removing unnecessary spaces**

In [ ]:
blog_df['text']=  blog_df['text'].str.replace('\s+',' ',regex = True)

In [ ]:
print(blog_df['text'][6])

 another classic from the leader of the free world our enemies are innovative and resourceful and so are we said george bush in an address yesterday they never stop thinking about new ways to harm our country and our people and neither do we this guy should be president 


In [ ]:
todo_df = pd.DataFrame()

In [ ]:
todo_df= blog_df[['text','target','len']]

In [ ]:
todo_df.head()

,text,target,len
0,the incompetence of transport planners i deal...,"[male, 30s, Government, Pisces]",354
1,2 10 04 one more week till huntsville yay and...,"[male, 10s, Student, Libra]",766
2,i woke up very early on saturday morning more...,"[female, 20s, indUnk, Aries]",692
3,ok lots more pictures urllink my apartment em...,"[male, 20s, Technology, Leo]",23
4,happy birthday mom today is mom birthday i kn...,"[male, 30s, indUnk, Aries]",92


In [ ]:
print(todo_df['text'][0])

 the incompetence of transport planners i deal a lot with public transport in my job and also use it a lot as everyone should this ensures minimal use of the old bmw normally m and i go in to torquay together in the car and i come home on the train because she stays on to do overtime or go to the dentist gym hairdresser while i come home at a reasonable time fridays we normally drive home together as she finishes at 4 15 on a friday a very wise and sensible time to finish if you ask me also one friday in 4 i give a lift to someone to a meeting in the afternoon and then take them home before picking m up which was what was scheduled for today also today m was having her hair done after work so i would have to repark the car after the meeting so she could pick it up and then i would catch the train home unfortunately having parked the car i realised that i didnt have time to walk to the station solution go to the pub and wait for the next train problem did not leave enough time to get to

In [ ]:
todo_df.describe()

,len
count,99534.0000000
mean,202.4929974
std,376.4966430
min,1.0000000
25%,38.0000000
50%,114.0000000
75%,257.0000000
max,51835.0000000


In [ ]:
ina = todo_df[todo_df['len']==0].index

In [ ]:
ina

Int64Index([], dtype='int64')

In [ ]:
todo_df.head()

,text,target,len
0,the incompetence of transport planners i deal...,"[male, 30s, Government, Pisces]",354
1,2 10 04 one more week till huntsville yay and...,"[male, 10s, Student, Libra]",766
2,i woke up very early on saturday morning more...,"[female, 20s, indUnk, Aries]",692
3,ok lots more pictures urllink my apartment em...,"[male, 20s, Technology, Leo]",23
4,happy birthday mom today is mom birthday i kn...,"[male, 30s, indUnk, Aries]",92


In [ ]:
todo_df.isna().sum()

text      0
target    0
len       0
dtype: int64

In [ ]:
todo_df.shape

(99534, 3)

In [ ]:
##Analyzing and Preparing Data

In [ ]:
max_features = 20000
maxlen = 100
embedding_size = 50

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words= 15000)

In [ ]:
tokenizer.fit_on_texts(list(todo_df['text']))
X = tokenizer.texts_to_sequences(todo_df['text'])

In [ ]:
print("number of samples:", len(X))
print(X[0])

number of samples: 99534
[2, 6, 6792, 1, 569, 5, 178, 21, 735, 6792, 9, 12, 281, 4, 126, 283, 7, 5, 178, 31, 202, 129, 22, 6942, 283, 6, 2, 193, 10769, 1749, 328, 4, 1, 63, 9, 3, 332, 9, 2, 291, 4, 1, 146, 133, 17, 2, 1112, 82, 48, 4690, 17, 3, 26, 7324, 43, 63, 3, 2, 4373, 1583, 142, 1, 146, 133, 29, 5, 3900, 54, 6764, 23, 1749, 534, 133, 332, 31, 48, 10071, 29, 204, 553, 17, 5, 414, 5, 108, 2131, 4, 7222, 54, 3, 832, 45, 13, 341, 18, 126, 42, 414, 9, 204, 1, 222, 5, 3098, 3, 176, 3, 5, 801, 9, 2, 723, 4, 57, 137, 79, 133, 140, 2332, 328, 37, 89, 14, 40, 14, 3610, 15, 93, 126, 93, 328, 14, 234, 51, 430, 232, 98, 111, 20, 1, 44, 16, 3, 2, 291, 98, 2, 801, 20, 48, 88, 598, 7, 37, 4, 57, 1, 44, 846, 2, 1112, 133, 1227, 234, 4719, 2, 291, 1, 2929, 8, 1, 631, 16, 54, 3, 479, 3, 2, 1170, 2876, 63, 3, 2, 3513, 4, 335, 15, 2, 163, 1112, 458, 70, 10, 338, 229, 54, 3, 50, 3, 2, 1170, 20, 234, 870, 2, 1112, 21, 5, 147, 335, 3, 2, 163, 1, 1126, 3, 269, 80, 3, 589, 17, 2, 775, 3, 846, 328, 21, 2, 

In [ ]:
##Padding the sequence

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X = pad_sequences(X , maxlen=maxlen)
X.shape

(99534, 100)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

In [ ]:
MB = MultiLabelBinarizer()

In [ ]:
y = MB.fit_transform(todo_df['target'])

In [ ]:
y

array([[0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 1, 0],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 0]])

In [ ]:
print(X.shape)
print(y.shape)

(99534, 100)
(99534, 57)


In [ ]:
len(tokenizer.word_index)## total number of words

230877

In [ ]:
## computing glove embedding

In [ ]:
text_file= "/content/drive/MyDrive/glove.6B.50d.txt"

In [ ]:
embedding = {}
for o in open(text_file):
  word = o.split(" ")[0]
  emb = o.split(" ")[1:]
  emb = np.asarray(emb, dtype= 'float32')
  embedding[word]=emb

In [ ]:
## Creating weight matrices

In [ ]:
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words,50))
for word , i in tokenizer.word_index.items():
  embedding_vector = embedding.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(230878, 50)

In [ ]:
## Splitting Training and Test

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size= 0.2)

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(79627, 100) (79627, 57)
(19907, 100) (19907, 57)


In [ ]:
#Model Building

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout,BatchNormalization
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
model = Sequential()
model.add(Embedding(num_words,embedding_size,weights=[embedding_matrix],input_length = maxlen, trainable=False))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(180,activation = 'relu'))
model.add(Dense(90,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(47,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(57,activation= 'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy' , optimizer= 'adam' , metrics = 'accuracy')

In [ ]:
model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           11543900  
_________________________________________________________________
flatten_36 (Flatten)         (None, 5000)              0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 5000)              20000     
_________________________________________________________________
dense_111 (Dense)            (None, 180)               900180    
_________________________________________________________________
dense_112 (Dense)            (None, 90)                16290     
_________________________________________________________________
dense_113 (Dense)            (None, 64)                5824      
_________________________________________________________________
batch_normalization_25 (Batc (None, 64)              

In [ ]:
history=model.fit(x_train,y_train,epochs=20,batch_size = 64, validation_data = (x_test,y_test),verbose=1)

Epoch 1/20
1245/1245 [==============================] - 24s 18ms/step - loss: 0.3844 - accuracy: 0.0935 - val_loss: 0.1693 - val_accuracy: 0.1535
Epoch 2/20
1245/1245 [==============================] - 23s 18ms/step - loss: 0.1706 - accuracy: 0.1810 - val_loss: 0.1687 - val_accuracy: 0.1652
Epoch 3/20
1245/1245 [==============================] - 23s 18ms/step - loss: 0.1684 - accuracy: 0.1875 - val_loss: 0.1687 - val_accuracy: 0.1599
Epoch 4/20
1245/1245 [==============================] - 23s 18ms/step - loss: 0.1656 - accuracy: 0.2122 - val_loss: 0.1690 - val_accuracy: 0.1676
Epoch 5/20
1245/1245 [==============================] - 23s 18ms/step - loss: 0.1615 - accuracy: 0.2258 - val_loss: 0.1711 - val_accuracy: 0.1511
Epoch 6/20
1245/1245 [==============================] - 23s 18ms/step - loss: 0.1561 - accuracy: 0.2293 - val_loss: 0.1757 - val_accuracy: 0.1759
Epoch 7/20
1245/1245 [==============================] - 22s 18ms/step - loss: 0.1505 - accuracy: 0.2282 - val_loss: 0.1841 -

In [ ]:
NN = model.predict(x_test)

In [ ]:
print(NN[0:5].round())

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [ ]:
print(y_test[0:5].round())

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]]


In [ ]:
print(classification_report(y_test,NN.round()))

              precision    recall  f1-score   support

           0       0.51      0.51      0.51      6886
           1       0.53      0.54      0.53      9443
           2       0.32      0.17      0.22      3578
           3       0.00      0.00      0.00       121
           4       0.00      0.00      0.00       136
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00      1456
           7       0.00      0.00      0.00        49
           8       0.00      0.00      0.00      1860
           9       0.00      0.00      0.00       966
          10       0.00      0.00      0.00        36
          11       0.00      0.00      0.00       123
          12       0.00      0.00      0.00        66
          13       0.00      0.00      0.00       143
          14       1.00      0.01      0.02      1963
          15       0.00      0.00      0.00      1410
          16       0.00      0.00      0.00       117
          17       0.00    

In [ ]:
Inference: 

*) Totally 57 classes out of which most are highly imbalanced classes.
*) Model is overfitting when the increasing the number of epochs. Training accruacy is 34% and test accuracy is 21%.
*) Using transfer learning with BERT model, LSTM layers may help us to increase the accuracy.

#ChatBot

In [20]:
import nltk
import string, random
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [123]:
import json

In [124]:
with open('/content/drive/MyDrive/GL Bot.json') as file:
  corpus = json.load(file)

In [125]:
print(corpus)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time', 'hey there', 'i need help'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', ' i have login issue with olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no li

Data Preparation

In [126]:
token = [] ## total words in the corpus
labels = [] ## target
tokenised_word = []
total_tags = []

for intent in corpus['intents']:
  for pattern in intent['patterns']:
    temp = nltk.word_tokenize(pattern)
    token.extend(temp)
    tokenised_word.append(temp)
    total_tags.append(intent['tag'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

In [127]:
len(token)

441

In [128]:
len(tokenised_word)

135

In [129]:
print(labels)

['Intro', 'Exit', 'Olympus', 'SL', 'NN', 'Bot', 'Profane', 'Ticket']


In [130]:
print(total_tags)

['Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Intro', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Exit', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'Olympus', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'Bot', 'Bot', 'Bot', 'Bot', 'Bot', 'Bot', 'Bot', 'Bot', 'Profane', 'Profane', 'Profane', 'Profane', 'Profane', 'Profane', 'Profane', 'Profane', 'Profane', 'Ticket', 'Ticket', 'T

** Applying the lemmetiser**

In [131]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [132]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [133]:
Train = []
Target = []

out_empty = [0 for _ in range(len(labels))]

for i,doc in enumerate(tokenised_word):
  bag = []

  w_stem = [lemmatizer.lemmatize(w.lower()) for w in doc] 

#Creating bag of words
  for w in token:
     if w in w_stem:
       bag.append(1)
     else:
       bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(total_tags[i])] = 1

  Train.append(bag) #List
  Target.append(output_row) #List

## Building Text Classification NN model

In [134]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [139]:
model = Sequential()
model.add(Flatten())
model.add(Dense(130,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(65,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(8,activation = 'softmax'))

In [140]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [141]:
checkpoint= ModelCheckpoint('/content/chatbot.h5',save_best_only=True, monitor='accuracy', 
                                                    mode='max', verbose=1)

In [142]:
model.fit(np.array(Train), np.array(Target), epochs=20, batch_size=15,callbacks=[checkpoint],verbose=1)

Epoch 1/20
9/9 [==============================] - 1s 4ms/step - loss: 2.7009 - accuracy: 0.1535

Epoch 00001: accuracy improved from -inf to 0.17778, saving model to /content/chatbot.h5
Epoch 2/20
9/9 [==============================] - 0s 3ms/step - loss: 1.7404 - accuracy: 0.3957

Epoch 00002: accuracy improved from 0.17778 to 0.42222, saving model to /content/chatbot.h5
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 1.2780 - accuracy: 0.5894

Epoch 00003: accuracy improved from 0.42222 to 0.57778, saving model to /content/chatbot.h5
Epoch 4/20
9/9 [==============================] - 0s 3ms/step - loss: 0.9693 - accuracy: 0.6915

Epoch 00004: accuracy improved from 0.57778 to 0.68148, saving model to /content/chatbot.h5
Epoch 5/20
9/9 [==============================] - 0s 3ms/step - loss: 0.7569 - accuracy: 0.7813

Epoch 00005: accuracy improved from 0.68148 to 0.77037, saving model to /content/chatbot.h5
Epoch 6/20
9/9 [==============================] - 0s 3ms/s

In [143]:
def tokenise_sentence(inp):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    temp = w_tokenizer.tokenize(inp)
    return temp

In [144]:
def bag_of_words(inp,token):
  w_tem = tokenise_sentence(inp)
  word = []
  bags = []
  w_stem = [lemmatizer.lemmatize(wor.lower()) for wor in w_tem] 
#Creating bag of words
  for w in token:
    if w in w_stem:
      bags.append(1)
    else:
      bags.append(0)
  word.append(bags)
  return(np.array(word))

In [145]:
def chat():
  print("chat with jason(type: quit to stop")
  print("If answer is not right (type:*)")
  while True:
    inp = input("\n\nYou: ")
    if inp.lower()=="*":
      print("Jason: Please rephrase your question and try again")
    if inp.lower()=="quit":
      print("Thank you! Bye")
      break
   
    result = model.predict(bag_of_words(inp,token))
    result_index = np.argmax(result)
    tag = labels[result_index]

    for tg in corpus["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
        
    print(random.choice(responses))

In [147]:
chat()

chat with jason(type: quit to stop
If answer is not right (type:*)


You: hello
Hello! how can i help you ?


You: have olympus login issue
Link: Olympus wiki


You: i hate you
Please use respectful words


You: not a good solution
Tarnsferring the request to your PM


You: what is your name
I am your virtual learning assistant


You: how logistic regression works
Link: Machine Learning wiki 


You: explain ensemble techniques
Link: Machine Learning wiki 


You: hyperparameter tunning
Link: Machine Learning wiki 


You: how to set hidden layers
Hello! how can i help you ?


You: hidden layers
Link: Neural Nets wiki


You: how gradient boosting works
Link: Machine Learning wiki 


You: how to operate olympus
Link: Olympus wiki


You: softmax layers
Link: Neural Nets wiki


You: you are stupid
Please use respectful words


You: complete jerk
Link: Machine Learning wiki 


You: thank you
I hope I was able to assist you, Good Bye


You: quit
Thank you! Bye


In [148]:
!jupyter nbconvert --to html AIML_R8_Project1_Word_Embeddings_with_DNN_&_Chatbot.ipynb

/bin/bash: _Chatbot.ipynb: command not found
[NbConvertApp] WARNING | pattern u'AIML_R8_Project1_Word_Embeddings_with_DNN_' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the res